## Question 1, Part 1

Determine reactivity insertion from localized flow blockage.

Na void positive reactivity due to spectral hardening

Less sodium density $\rightarrow$ less scattering $\rightarrow$ more fast neutrons $\rightarrow$ less absorption

![image](img/Pu_239_cross_section.png)

Operative word is "Local"

* Void worth $4.44 assuming "flowing sodium completely voided in ALL active and above-core regions" ([Argonne's Fast Reactor Physics - 2](https://www.nrc.gov/docs/ML1914/ML19148A801.pdf))
* How to apply this for local voiding?
* Ultimately want reactivty as a function of sodium density, i.e. $k(\rho_{Na})$

Driven by scattering out of fast spectrum
Assumptions to derive $k(\rho_{Na})$
* Assume two group equation
* Infinite reactor (or perfect reflector)
* All relevant fissions occur in fast spectrum (terrible assumption...)
* Everything scattered out of fast spectrum is absorbed (also pretty bad)

## Two group equation
Group 1 - fast spectrum

Group 2 - everything else
$$
\frac{1}{k}\nu_1 \Sigma_f \phi_1 - \Sigma_{a1} \phi_1 - \Sigma_{1\rightarrow 2}\phi_1 = 0 \\
\Sigma_{1 \rightarrow 2} \phi_1 - \Sigma_{a2} \phi_2  = 0
$$

If we can generate a linear expression for $k(\rho_{Na})$, we can use what know about global void worth to solve for linear constants.

$$
k(\Sigma_{a1} + \Sigma_{1 \rightarrow 2}) = \nu_1 \Sigma_{f1}
$$
$$
k\left(\frac{\Sigma_{a1}}{\nu_1 \Sigma_{f1}} + \frac{\Sigma_{1 \rightarrow 2}}{\nu_1 \Sigma_{f1}}\right) = 1
$$
$$
k\left(\frac{\Sigma_{a1}}{\nu_1 \Sigma_{f1}} + \frac{\rho_{Na}A_v}{M_{Na}}\frac{\sigma_{1 \rightarrow 2}}{\nu_1 \Sigma_{f1}}\right) = 1
$$
$$
\begin{equation}
k\left(A + B\rho_{Na}\right) = 1 
\end{equation}
$$


We have two known reactor conditions that we can now use to solve for A and B in the above equation:
1. Void worth when entire core is voided ($4.44 BOC, metal, startup)
2. Assume k = 1 when entire core is liquid

For Na density, we will use (from [ANL-RE-95-2](https://www.ne.anl.gov/eda/ANL-RE-95-2.pdf))
1. Density of vapor sodium at saturation ($\sim 1200 K$ at operating pressure from [Figure III.7-12](https://publications.anl.gov/anlpubs/2017/04/134264.pdf)) is $0.394 \frac{kg}{m^3}$.
2. Density of liquid sodium at core average temperature $700 K$ (average of $350^o C$ cold pool temperature and $500^o C$ hot temperature) is $852 \frac{kg}{m^3}$.

This results in a 2x2 linear system that we can use to solve for A and B:
$$
\begin{align} \tag{2}
Ak_g + Bk_g\rho_{na,g} = 1 \\
Ak_l + Bk_l\rho_{na,l} = 1
\end{align}
$$

$$
\begin{bmatrix}
k_g & k_g \rho_{na,g} \\
k_l & k_l \rho_{na,l} \\
\end{bmatrix}

\begin{bmatrix}
A \\ B
\end{bmatrix} 

=

\begin{bmatrix}
1 \\ 1
\end{bmatrix}
$$

In [136]:
"""Let's set up some units before we get started.

And a general note on comments since you'll want to know how I
work. My rule of thumb is to write comments according to my audience.
In many cases, my audience is engineers, and therefore my comments
are extensive. But comments, like lines of code, represent inventory
of an application. As your inventory goes up, your maintenance costs
increase as well. Comments require maintenance just like source code
in the case of refactoring. If the code is obvious or the audience is
savvy, comments oftentimes are not necessary. However, I'll be using
lots of comments in this project assuming my audience is primarily engineers,
plus the fact that computational code is confusing and hard to read.

Most of the code I write is in standalone applications rather than Jupyter
notebooks, so I'll use google's docstring guide inside of some code blocks
where appropriate (which is sort of what you see here).

Attributes:
    ureg (pint.UnitRegistry): Pint unit registry
    Quantity (pint.Quantity): Pint float quantity containing units
"""
import pint
from math import floor, log10

# I've had success using pint to keep me from making dumb unit errors.
# So I'll try not to embarass myself in front of yall too badly.
ureg = pint.UnitRegistry()
Quant = pint.Quantity

def round_quant(quantity, sig_fig=4):
    """Return pint quantity reduced to given significant digits as a string."""
    value = round(
        quantity.m, 
        sig_fig-int(floor(log10(abs(quantity.m)))) - 1
    )
    return str(value) + str(quantity.units)


In [137]:
"""Solving for A and B in equation 1.

Assumes metal startup core at BOC. k_1, rho_1, k_2, and rho_2
from Equation 2 are first determined using the assumptions 
outlined above. They become the coefficients for the 2x2 system
used to solve for A and B.

Attributes:
    global_void_reactivity (float): Whole core void reactivity
    k_g (float): Multiplication factor with entire core voided
    k_l (float): Reactor critical multiplication factor or 1.0
    rho_na_g (Quantity): Vapor density of sodium
    rho_na_l (Quantity): Liquid density of sodium
    A (float): Constant from Equation 1
    B (float): Constant from Equation 1
"""
import numpy as np

# k_g from Equations 2 is solved from the void worth 
# of the entire core
void_worth = 4.44 # Dollars
beta_eff = 0.0034 # Delayed neutron fraction
global_void_reactivity = void_worth * beta_eff
k_g = 1 / (1 - global_void_reactivity)
# k2 is assumed to be 1
k_l = 1.0

# Density of voided sodium
rho_na_g = Quant(0.394, "kg/m^3")
# Density of liquid sodium
rho_na_l = Quant(852, "kg/m^3")

# Solving the 2x2 system for coefficients A and B in Equation 1
# 2x2 coefficient matrix
coeffs = np.array([[k_g, k_g*rho_na_g.m], [k_l, k_l*rho_na_l.m]])
# Right hand side term
rhs = np.array([1, 1])
# Solving for constants A and B from Equation 1
A,B = np.linalg.solve(coeffs, rhs)

Now we can return reactivity as a function of core sodium void fraction. 
This is my interpretation of "Local" in question 1, part 1 for the miniproject.

In [138]:
def void_reactivity(alpha_na):
    """Calculate the reactivity insertion of ABR due to Na voiding.
    
    Using the void worth of the active whole-core, calculate the
    reactivity insertion from a fraction of void within a single
    assembly using the following relationships:
        * reactivity = k - 1 / k
        * k = 1 / (A + B*rho_na) from Equation 1
        * rho_na = alpha_na*rho_na_g + (1- alpha_na)*rho_na_l 

    Args:
        alpha_na (float): Sodium void fraction of ABR core. 
            1.0 = Entire active core is vapor
            0.0 = Entire active core is liquid

    Returns:
        float: Reactivity insertion as a function of Na void fraction
    """
    # This function is only valid for alpha_na between 1 and 0, so 
    # lets do some raisen if this isn't the case.
    if alpha_na > 1.0 or alpha_na < 0.0:
        raise ValueError(
            "Sodium volume fraction must be between 1.0 and 0.0. "
            f"Received a value of {alpha_na}."
        )
    
    # Turn alpha_na to a float because Pint doesn't like float64s
    alpha_na = float(alpha_na)

    # Core sodium density
    rho_na = alpha_na*rho_na_g + (1-alpha_na)*rho_na_l

    # Multiplication factor
    k = 1 / (A + rho_na.m*B)

    # Return reactivity
    return (k-1)/k
    


In [139]:
"""Test void reactivity equaiton.

This of course goes in a test module, but I figured it would
be good to have it here as well. Assertions are based on the
assumptions for a voided core and a liquid core.
"""
from glob import glob
import pytest

# Make sure exceptions are handled appropriately
with pytest.raises(ValueError) as err:
    # The Answer s way too big!
    void_reactivity(42)
# I always check my error statements since you could raise for
# the wrong reason.
assert "Received a value of 42" in str(err.value)

with pytest.raises(ValueError) as err:
    # Negative The Answer?
    void_reactivity(-42)
assert "Received a value of -42" in str(err.value)

# Check for liquid solid core
assert void_reactivity(0.0) == 0.0
# Check for vapor solid core
assert void_reactivity(1.0) == pytest.approx(global_void_reactivity)

It's a bummer you don't get green dots from assertions in Jupyter notebooks like with pytest. I think I'm addicted to those things.

In [140]:
"""Pretty picture time."""
import plotly.graph_objects as go

x_values = np.linspace(0.0, 1.0, num=20)  # volume fraction
y_values = [void_reactivity(i) for i in x_values]  # reactivity

fig = go.Figure(
    data=[go.Scatter(x=x_values, y=y_values, mode="lines")],
)
fig.update_layout(
    title="Reactivity insertion as a function of Na void fraction.",
    xaxis_title="Na void fraction",
    yaxis_title="Reactivity"
)

fig.show()

This is kind of boring
And we all knew this would just be a linear function of $\alpha_{Na}$.
More complex neutronics relationships can improve the accuracy of Equation 1.
* Actually consider thermal fission
* More energy groups
* Heterogeneity

A subchannel code (e.g. COBRA) could be used to generate the local void fraction used in the above python function `void_reactivity`.
As it is I've spent a lot of time on this (fairly simple) derivation and won't try and build a subchannel solver right now :).


## Question 1, Part 2
How hot do things get for a Na void reactivity insertion?

In [141]:
"""Reactor constants we'll need for this part.

Again assumes startup and metal core at BOC.

Attributes:
    reactor_power (pint.Quantity): ABR thermal reactor power, 1000MW_th
    l_p (pint.Quantity): ABR prompt neutron lifetime

"""
reactor_power = Quant(1000, "MW")  # ABR thermal reactor power
l_p = Quant(0.37, "microseconds") # Prompt neutron lifetime
tau = Quant(13.05, "s") # Weighted delayed neutron decay constant



In [142]:
def void_reactor_power(alpha_na=0.0, time=Quant(0.0, "s")):
    """Calculate reactor power at a given time due to Na voiding.

    Using point kinetics and the reactivity insertion due to voiding,
    calculate reactor power after a given time. Point kinetics equation
    uses the mean generation time of delayed and fast neutrons, l_d,
    with only one group of delayed neutrons.
    These equations are
        * l_d = (1-beta)l_p + tau*beta
        * P(t) = P(0) * exp((k-1)/l_d * t)
    
    Args:
        alpha_na (float): Sodium void fraction of ABR core.
            1.0 = Entire active core is vapor
            0.0 = Entire active core is liquid
        time (pint.Quantity, float): Time in seconds at which to calculate reactor power.
    """
    # If time is not a quantity, assume given in seconds
    if not isinstance(time, pint.Quantity):
        time = Quant(time, 's')

    # Calculate the mean neutron generation lifetime
    l_d = (1-beta_eff)*l_p + tau*beta_eff

    # Multiplication factor is given by void_reactivity expression
    k = 1 / (1 - void_reactivity(alpha_na))

    # Calculate reactor period and validate unitless
    period = l_d / (k - 1)
    
    # Calculate reactor power at provided time
    return reactor_power*np.exp((time.to("s")/period.to("s")).m)


rather than solutions to traditional point reactor kinetics equations.
If I have time, I'll loop back and solve a simple version in a time
loop because that actually sounds pretty fun! But I've spent a 

In [143]:
"""Testing block for void_reactor_power."""
assert void_reactor_power(alpha_na=1.0, time=0.0) == reactor_power
# Just ignore the divide by zero...totally normal
assert void_reactor_power(alpha_na=0.0, time=Quant(1000.0, 'months')) == reactor_power

In [164]:
"""Pretty 3D pictures!"""
alpha_array = np.linspace(0.0, 1.0, num=20)
time_array = np.linspace(0.0, 5, num=20)
power_surface = np.empty([alpha_array.size, time_array.size])

# Loop over the void and time range to fill in the power surface
for i in range(alpha_array.size):
    for j in range(time_array.size):
        power_surface[i][j] = void_reactor_power(alpha_array[i], time_array[j]).m

fig = go.Figure(
        data=go.Surface(
            x=time_array, 
            y=alpha_array, 
            z=power_surface,
            contours = {
                'x': {'show': True, 'start': 0, 'end': 5, 'size': 0.5},
                'y': {'show': True, 'start': 0, 'end': 1, 'size': 0.1}
        }
        )
)
fig.update_layout(
    title="Reactor power as a function of Na void fraction and time.",
    scene = dict(
        xaxis_title="Time (s)",
        yaxis_title="Na void",
        zaxis_title="Power (MW)",
    )
)
fig.show()

Temperature increase is calculated using 
$$
\dot{q} = \dot{m}c_p \Delta T \\
\Delta T = \frac{\dot{q}}{\dot{m} c_p}
$$

Only consider temperature rise of the Na coolant (this assumption is a bit painful for me).

For $\dot{m}$, use mass flow of assembly with peak cladding temp.
Peak cladding temp = $631.9^oC$
Mass flow of limiting assembly = $29.6 kg/s$

![image](img/peak_cladding_temps.png)
![image](img/mass_flow_rates.png)


For $\dot{q}$, use `void_reactor_power` and the power distribution of the limiting assembly $\dot{q}_{asmbly} = \frac{6.11MW}{1000MW} \dot{q}\left(\alpha_{Na}, t\right)$

![image](img/power_distributions.png)

For a local blockage (likely in the orifice plate?), assume only one assembly will void.

In [165]:
"""Maximum void fraction for local blockage.

Assumes only one assembly of the core region of ABR will void.
"""
total_assemblies = 180 # Total core assemblies
voided_assemblies = 1 # Assume only one voides for local blockage
alpha_na_max = voided_assemblies / total_assemblies

In [166]:
"""Specific heat of Na"""
cp_na_g = Quant(2.51, "kJ/kg/K") # Vapor at 1200K
cp_na_l = Quant(1.277, "kJ/kg/K") # Liquid at 700K

In [167]:
"""Limiting assembly values."""
# Mass flow for limiting assembly
mdot = Quant(29.6, "kg/s")

# Power fraction of the limiting channel
power_frac = Quant(6.11, "MW")/Quant(1000, "MW")


In [168]:
def void_temp_rise(alpha_na=0.0, time=Quant(0.0, "s")):
    """Assembly temperature rise for a local blockage.

    Calculate the temperature rise after a given period of time
    of a single assembly that experiences a local blockage.
    Assumes only sodium is absorbing heat.

    Args:
        alpha_na (float): Sodium void fraction of ABR core.
            1.0 = Entire active core is vapor
            0.0 = Entire active core is liquid
        time (pint.Quantity, float): Time in seconds at which to calculate reactor power.
        
    Returns:
        (pint.Quantity): Temperature rise in K
    """
    # If time is not a quantity, assume given in seconds
    if not isinstance(time, pint.Quantity):
        time = Quant(time, 's')

    # Calculate the specific heat of Na using the volume fraction
    cp_na = alpha_na*cp_na_g + (1-alpha_na)*cp_na_l

    # Calculate the temperature rise
    return (power_frac*void_reactor_power(alpha_na, time)/mdot/cp_na).to("K")


In [169]:
print(void_temp_rise(0.0, 0.0))
print(void_temp_rise(0.0, 5))

161.6436326694745 kelvin
161.6436326694745 kelvin


In [170]:
"""Testing block for void temperature rise."""
assert void_temp_rise(0.0, 0.0) == (power_frac*reactor_power/mdot/cp_na_l).to("K")

In [171]:
"""Last pretty picture."""
temp_surface = np.empty([alpha_array.size, time_array.size])

# Loop over the void and time range to fill in the power surface
for i in range(alpha_array.size):
    for j in range(time_array.size):
        temp_surface[i][j] = void_temp_rise(alpha_array[i], time_array[j]).m

fig = go.Figure(
    data=go.Surface(
        x=time_array, 
        y=alpha_array,
        z=temp_surface,
        contours = {
            'x': {'show': True, 'start': 0, 'end': 5, 'size': 0.5},
            'y': {'show': True, 'start': 0, 'end': 1, 'size': 0.1}
        }
    )
)
fig.update_layout(
    title="Limiting assembly Delta-T as a function of Na void fraction and time.",
    scene = dict(
        xaxis_title="Time (s)",
        yaxis_title="Na void",
        zaxis_title="Delta T (K)"
    )
)
fig.show()
